In [1]:
import pandas
pandas.set_option("display.max_rows", 50, "display.max_columns", 50)

import numpy
import pygsheets

import Pipelines_Current dataset

In [2]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
#spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')
spreadsheet = gc.open_by_key('178YQvZ8EC17LYA8sIgsOEi5veROivWi_uUnPb6tOuzE')

#spreadsheet[1] "Gas Pipelines" tab is the second index
gas_pipes = spreadsheet[1].get_as_df(encoding='latin1')
oil_pipes = spreadsheet[2].get_as_df(encoding='latin1')
#owners = spreadsheet[2].get_as_df()

gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
oil_pipes = oil_pipes.drop('WKTFormat', axis=1)
pipes_df = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#pipe = spreadsheet[1].get_as_df()
#pipe = pipe.drop('WKTFormat', axis=1)

#get other relevant sheets
country_ratios = spreadsheet[5].get_as_df(encoding='latin1')
owners_df_orig = spreadsheet[3].get_as_df(encoding='latin1')
owners_df_orig.set_index('ProjectID', inplace=True)

In [3]:
country_ratios.replace('--', numpy.nan, inplace=True)

owners_df_orig.replace('',numpy.nan,inplace=True)
owners_df_orig.replace('--',numpy.nan,inplace=True)

### remove petrobras Routes b,c pipelines in Brazil (overcounting otherwise)

In [17]:
routes_to_zero_out = [
    'Route 4b Gas Pipeline',
    'Route 5b Gas Pipeline',
    'Route 5c Gas Pipeline',
    'Route 6b Gas Pipeline' ]

cols_to_zero_out = ['LengthMergedKm']
owners_df_orig.loc[owners_df_orig['PipelineName'].isin(routes_to_zero_out), cols_to_zero_out] = numpy.nan

In [18]:
# get a set list of all owners
owner_column_list = ['Owner1', 
                     'Owner2', 
                     'Owner3', 
                     'Owner4', 
                     'Owner5', 
                     'Owner6', 
                     'Owner7', 
                     'Owner8', 
                     'Owner9', 
                     'Owner10', 
                     'Owner11']

percent_column_list = ['Percent1', 
                       'Percent2', 
                       'Percent3', 
                       'Percent4', 
                       'Percent5', 
                       'Percent6', 
                       'Percent7', 
                       'Percent8', 
                       'Percent9', 
                       'Percent10', 
                       'Percent11']

owner_id_list = ['ID1', 
                 'ID2', 
                 'ID3', 
                 'ID4', 
                 'ID5', 
                 'ID6', 
                 'ID7', 
                 'ID8', 
                 'ID9', 
                 'ID10', 
                 'ID11']

### sum MergedKmByCountry and MergedKmByRegion

In [19]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(list(set(country_ratios['Country'])))
region_list = sorted(list(set(country_ratios['Region'])))

In [20]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']

In [21]:
fuel_type = 'Gas'
#fuel_type = 'Oil'
#fuel_type = 'NGL'

country_ratios_subset = country_ratios[country_ratios['Fuel']==fuel_type]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_subset_status = country_ratios_subset[country_ratios_subset['Status']==status]
    km_by_country[status] = country_ratios_subset_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_subset_status = country_ratios_subset[country_ratios_subset['Status']==status]
    km_by_region[status] = country_ratios_subset_status.groupby('Region')['MergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['In Development (Proposed + Construction)'] = km_by_region[['Proposed','Construction']].sum(axis=1)
km_by_country['In Development (Proposed + Construction)'] = km_by_country[['Proposed','Construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.name = 'Region'
km_by_country.index.name = 'Country'

km_by_region.to_excel(fuel_type+'KmByRegionAndStatus.xlsx')
km_by_country.to_excel(fuel_type+'KmByCountryAndStatus.xlsx')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


### create an excel file with all unique owners, not specific to fuel type

In [22]:
owner_list = []
for column in owner_column_list:
    owner_list+=list(owners_df_orig[column])
owner_list = list(set(owner_list))
# remove empty (no owner)
owner_list.remove(numpy.nan)
unique_owner_list = sorted(owner_list)

#pandas.Series(unique_owner_list).to_excel('AllFuels'+'UniqueOwnersList.xlsx')

## pipeline km by parent company (owner) and project status

In [23]:
fuel_type = 'Gas'
#fuel_type = 'Oil'
#fuel_type = 'NGL'

owners_df_subset = owners_df_orig.copy()[owners_df_orig['Fuel']==fuel_type]

##################################################

owner_list_subset = []
for column in owner_column_list:
    owner_list_subset+=list(owners_df_subset[column])
owner_list_subset = list(set(owner_list_subset))
# remove empty (no owner)
owner_list_subset.remove(numpy.nan)
unique_owner_list_subset = sorted(owner_list_subset)

##################################################
# convert percents to fractions
##################################################
owners_df_fractions = owners_df_subset.copy()

for col in percent_column_list:
    owners_df_fractions[col] = owners_df_fractions[col].apply(lambda x: numpy.nan if x in [numpy.nan] 
                                          else x[:-1]).astype(float)/100
#df['col'] = df['col'].apply(lambda x: np.nan if x in ['-'] else x[:-1]).astype(float)/100

##################################################
# create km count by owner, status
##################################################
no_owner_info_count = 0
owners_km_by_status_df = pandas.DataFrame(0, index=unique_owner_list_subset, columns=status_list)

for status in status_list:
    
    owners_df_temporary = owners_df_fractions.copy()[owners_df_fractions['Status']==status]
    
    for idx,row in owners_df_temporary.iterrows():
        
        ### how many owners are there?
        row_owners = list(row[owner_column_list])
        row_owners = [i for i in row_owners if str(i)!='nan']
        n_owners = row_owners.__len__()
        row_fractions = list(row[percent_column_list])
        row_fractions = [i for i in row_fractions if str(i)!='nan']

        ### if there are no owners listed, continue to next loop iteration
        if n_owners==0:
            no_owner_info_count+=1
            continue # doesn't complete the rest of the ifs

        ### now if row fractions is an empty list, create equal fractions list instead
        if row_fractions==[]:
            row_fractions = [1/n_owners]*n_owners

        pipe_length = row['LengthMergedKm']
        owner_km_fractions = numpy.array(row_fractions)*pipe_length

        for owner_idx in range(n_owners):
            owners_km_by_status_df.loc[row_owners[owner_idx]][status]+=owner_km_fractions[owner_idx]

owners_km_by_status_df.index.name = 'Parent Company'
owners_km_by_status_df['In Development (Proposed + Construction)'] = owners_km_by_status_df[['Proposed','Construction']].sum(axis=1)
owners_km_by_status_df = owners_km_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_km_by_status_df = owners_km_by_status_df[excel_status_list]
owners_km_by_status_df.to_excel(fuel_type+'KmByOwnerAndStatus.xlsx')

owners_km_by_status_df_sorted = owners_km_by_status_df.sort_values('In Development (Proposed + Construction)', ascending=False)
owners_km_by_status_df_sorted.to_excel(fuel_type+'KmByOwnerAndStatus-SortedDescending.xlsx')

### number of developers globally

In [24]:
owners_km_by_status_df_sorted[
    owners_km_by_status_df_sorted['In Development (Proposed + Construction)']!=0].shape

(251, 9)

In [25]:
owners_km_by_status_df_sorted[:20]['In Development (Proposed + Construction)'].sum() / owners_km_by_status_df_sorted['In Development (Proposed + Construction)'].sum()

0.5790763161812137

In [26]:
owners_km_by_status_df_sorted[:6]['In Development (Proposed + Construction)'].sum() / owners_km_by_status_df_sorted['In Development (Proposed + Construction)'].sum()

0.3554681685108917

### perc increase for Australia

In [27]:
km_by_country.loc['Australia']['In Development (Proposed + Construction)']

12798.11

In [28]:
km_by_country.loc['Australia']['Operating']

28436.43

In [29]:
12200/39000

0.3128205128205128